# מטלת סיכום חלק 1

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
def get_cars(keywords):
    car_list = list()
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.ad.co.il/car?keyword=" + keywords
    response = requests.get(url)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        results_page = BeautifulSoup(response.content, 'html.parser')
        cars = results_page.find_all('div',{'class':'card-body p-md-3'})
        for car in cars:
            car_link = "https://www.ad.co.il" + car.find('a').get('href')
            car_name = car.find('a').get_text()
            
            if keywords in car_name:
                car_list.append((car_link)) 
        return car_list
    except:
        return None

In [3]:
get_cars("אלפא רומיאו")

['https://www.ad.co.il/ad/16027552',
 'https://www.ad.co.il/ad/15941395',
 'https://www.ad.co.il/ad/16108801',
 'https://www.ad.co.il/ad/14706603',
 'https://www.ad.co.il/ad/14671215',
 'https://www.ad.co.il/ad/14655354',
 'https://www.ad.co.il/ad/14530834',
 'https://www.ad.co.il/ad/14500119',
 'https://www.ad.co.il/ad/14198876',
 'https://www.ad.co.il/ad/14108320',
 'https://www.ad.co.il/ad/13939340',
 'https://www.ad.co.il/ad/13815766',
 'https://www.ad.co.il/ad/13742668',
 'https://www.ad.co.il/ad/15885588',
 'https://www.ad.co.il/ad/15761625',
 'https://www.ad.co.il/ad/14439006',
 'https://www.ad.co.il/ad/14348853',
 'https://www.ad.co.il/ad/15761645']

In [4]:
def preprocess_date_string(date_str):
    # Check if the input is a valid string or NaT
    if pd.isna(date_str):
        return "nan"
        
    # Check for slash or hyphen delimiters
    if '/' in date_str:
        parts = date_str.split('/')
    elif '-' in date_str:
        parts = date_str.split('-')
    else:
        return np.nan
    
    # Ensure parts are in the form of [YY, MM, DD] or [YY, MM]
    if len(parts) == 2:  # Format: YY/MM or YY-MM
        month, year = parts
        day = '01'  # Default day to the first of the month
    elif len(parts) == 3:  # Format: YY/MM/DD or YY-MM-DD
        day, month, year = parts
    else:
        return np.nan
    
    # Ensure all parts have leading zeros if necessary
    year = year.zfill(2)
    month = month.zfill(2)
    day = day.zfill(2)
    
    # Assume any two-digit year 'YY' represents '20YY'
    if len(year) == 2:
        year = '20' + year
    
    return f'{year}-{month}-{day}'

def days_until(date):
    today = datetime.today()
    delta = date - today
    return delta.days

In [5]:
def get_car_info(car_link):
    car_dict = {}
    try:
        response = requests.get(car_link)
        if response.status_code != 200:
            return {"error": "Failed to retrieve page"}
        
        result_page = BeautifulSoup(response.content, 'html.parser')
        meta_element = result_page.find('meta', itemprop='position', content='3')
        if meta_element:
            li_element = meta_element.find_parent('li', class_='breadcrumb-item')
            if li_element:
                item_name = li_element.find('span', itemprop='name')
                if item_name:
                    name = item_name.get_text()
                    car_dict["manufactor"] = name
        
        table = result_page.find('table', {'class': 'table table-sm mb-4'})
        if table:
            tbody = table.find('tbody')
            if tbody:
                for tr in tbody.find_all("tr"):
                    td = tr.find_all('td')
                    if len(td) >= 2:
                        key = td[0].get_text().strip()
                        value = td[1].get_text().strip()
                        car_dict[key] = value
        
        pictures = result_page.find_all('div', {'class': 'justify-content-center px-1'})
        car_dict['Pic_num'] = len(pictures)
        
        model_and_price = result_page.find_all('h2', {'class': 'card-title'})
        if len(model_and_price) >= 2:
            car_dict['Price'] = model_and_price[1].get_text().strip().replace('₪', '').replace(',', '').replace(' ', '')
            car_dict['model'] = model_and_price[0].get_text().strip()
        
        Repub_date_and_Cre_date = result_page.find_all('div', {'class': 'px-3'})
        if len(Repub_date_and_Cre_date) >= 2:
            cre_date = Repub_date_and_Cre_date[0].get_text().split()
            Repub_date = Repub_date_and_Cre_date[1].get_text().split()
            if len(cre_date) >= 3:
                car_dict['Cre_date'] = cre_date[2]
            if len(Repub_date) >= 4:
                car_dict['Repub_date'] = Repub_date[3]
        description_tag = result_page.find('p', {'class': 'text-word-break'})
        if description_tag:
            car_dict['description'] = description_tag.get_text().strip()
        
        if 'שנה' in car_dict:
            car_dict['שנה'] = int(car_dict['שנה'])
        if 'יד' in car_dict:
            car_dict['יד'] = int(car_dict['יד'])
        if 'נפח' in car_dict:
            car_dict['נפח'] = int(''.join(filter(str.isdigit, car_dict['נפח'])))
        if 'Price' in car_dict:
            car_dict['Price'] = float(car_dict['Price'])
        if 'Cre_date' in car_dict:
            car_dict['Cre_date'] = datetime.strptime(car_dict['Cre_date'], '%d/%m/%Y').date()
        if 'Repub_date' in car_dict:
            car_dict['Repub_date'] = datetime.strptime(car_dict['Repub_date'], '%d/%m/%Y').date()
        if 'ק"מ' in car_dict:
            car_dict['ק"מ'] = int(car_dict['ק"מ'].replace(',', ''))
        if 'טסט עד' in car_dict:
            test_date_str = preprocess_date_string(car_dict['טסט עד'])
            test_date = pd.to_datetime(test_date_str, errors='coerce')
            if not pd.isna(test_date):
                car_dict['days_until_test'] = int(days_until(test_date))
            del car_dict['טסט עד']
        
        return car_dict
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return {"error": str(e)}

In [6]:
get_car_info('https://www.ad.co.il/ad/16108801')

{'manufactor': 'אלפא רומיאו',
 'שנה': 2023,
 'יד': 1,
 'ת. הילוכים': 'אוטומטית',
 'נפח': 2000,
 'סוג מנוע': 'בנזין',
 'ק"מ': 5000,
 'צבע': 'לבן',
 'בעלות קודמת': 'פרטית',
 'בעלות נוכחית': 'פרטית',
 'אזור': 'גליל ועמקים',
 'עיר': 'נצרת',
 'Pic_num': 6,
 'Price': 250000.0,
 'model': "אלפא רומיאו ג'וליה",
 'Cre_date': datetime.date(2024, 2, 25),
 'Repub_date': datetime.date(2024, 2, 25),
 'description': 'רכב הכי מצוין',
 'days_until_test': 210}

In [7]:
def get_all_cars(keywords):
    results = list()
    all_cars = get_cars(keywords)
    for car in all_cars:
        x = get_car_info(car)
        results.append(x)
    return results

In [8]:
alfa_cars=get_all_cars("אלפא רומיאו")

In [9]:
alfa_cars

[{'manufactor': 'אלפא רומיאו',
  'שנה': 2015,
  'יד': 3,
  'ת. הילוכים': 'אוטומטית',
  'נפח': 1800,
  'סוג מנוע': 'בנזין',
  'ק"מ': 180000,
  'צבע': 'שחור',
  'בעלות קודמת': 'פרטית',
  'בעלות נוכחית': 'פרטית',
  'אזור': 'חיפה וחוף הכרמל',
  'עיר': 'חיפה',
  'Pic_num': 10,
  'Price': 80000.0,
  'model': "אלפא רומיאו ג'ולייטה",
  'Cre_date': datetime.date(2023, 11, 23),
  'Repub_date': datetime.date(2023, 11, 23),
  'description': 'אלפא רומיאו ג׳ולייטה תלתן ירוק שנת 2015 הכי מיוחדת בארץ.\nלאנץ אדישן (1/999)\nגמיש מעט\nאחרי יישור קו: קלאצ׳ים חדשים, משאבת מים, טיימינג, ציריות, תושבות, בושינגים למשולשים ועוד… האוטו מכנית מושלם. כל הרכב עיטוף PPF בשווי 20K₪. הרכב משופר עומד על בערך 350 כוח סוס, סטינג 2 +, קוילאוברים KW V2, דיפרנציאל מוגבל החלקה LSD, קיט יניקה GSR, רפידות, צלחות, אינטרקולר מוגדל לטורבו, מוטות מייצבים קדמיים ואחוריים, כיסאות באקט, חלקי קרבון, תאורה ראשית טורבו לד לבן, מערכת מולטימדיה של pioneer עם Car Play אלחוטי, ועוד המון תוספות….'},
 {'manufactor': 'אלפא רומיאו',
  'שנה': 2

In [10]:
import pandas as pd
df=pd.DataFrame(alfa_cars)


In [11]:
df = df.rename(columns={'שנה': 'Year', 'יד': 'Hand','ת. הילוכים': 'Gear', 'נפח': 'Engine_capacity','סוג מנוע': 'Engine_type', 'ק"מ': 'Km','טסט עד': 'Test ', 'צבע': 'Color','עיר': 'City', 'אזור': 'Area','בעלות נוכחית':'Prev_ownership','בעלות קודמת':'Curr_ownership'})

In [12]:
df['Cre_date'] = pd.to_datetime(df['Cre_date'], dayfirst=True)
df['Gear'] = pd.Categorical(df['Gear'])
df['Km'] = pd.to_numeric(df['Km'], errors='coerce').fillna(0).astype(int)
df['Engine_type'] = pd.Categorical(df['Engine_type'])
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'])
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'])
df['Area'] = df['Area'].astype(str)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0).astype(float)
df['Repub_date'] = pd.to_datetime(df['Repub_date'], dayfirst=True)
df['days_until_test'] = pd.to_numeric(df['days_until_test'], errors='coerce').replace(0, np.nan).astype('Int64')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   manufactor       18 non-null     object        
 1   Year             18 non-null     int64         
 2   Hand             18 non-null     int64         
 3   Gear             18 non-null     category      
 4   Engine_capacity  18 non-null     int64         
 5   Engine_type      18 non-null     category      
 6   Km               18 non-null     int32         
 7   Color            10 non-null     object        
 8   Curr_ownership   10 non-null     category      
 9   Prev_ownership   10 non-null     category      
 10  Area             18 non-null     object        
 11  City             18 non-null     object        
 12  Pic_num          18 non-null     int64         
 13  Price            18 non-null     float64       
 14  model            18 non-null     object     

In [14]:
df

,manufactor,Year,Hand,Gear,Engine_capacity,Engine_type,Km,Color,Curr_ownership,Prev_ownership,Area,City,Pic_num,Price,model,Cre_date,Repub_date,description,days_until_test
0,אלפא רומיאו,2015,3,אוטומטית,1800,בנזין,180000,שחור,פרטית,פרטית,חיפה וחוף הכרמל,חיפה,10,80000.0,אלפא רומיאו ג'ולייטה,2023-11-23,2023-11-23,אלפא רומיאו ג׳ולייטה תלתן ירוק שנת 2015 הכי מי...,<NA>
1,אלפא רומיאו,2017,3,אוטומטית,2000,בנזין,93000,אדום מטאלי,פרטית,פרטית,גליל ועמקים,שפרעם,4,99.0,אלפא רומיאו ג'וליה,2023-09-27,2023-09-27,שמורה בקנאות טיפולים בחברה \nחלונות שחורים ספו...,<NA>
2,אלפא רומיאו,2023,1,אוטומטית,2000,בנזין,5000,לבן,פרטית,פרטית,גליל ועמקים,נצרת,6,250000.0,אלפא רומיאו ג'וליה,2024-02-25,2024-02-25,רכב הכי מצוין,210
3,אלפא רומיאו,2015,3,אוטומטית,1800,בנזין,147000,אדום,פרטית,פרטית,באר שבע והסביבה,באר שבע,6,70000.0,אלפא רומיאו ג'ולייטה,2022-11-07,2022-11-07,אלפא רומיאו גולייטה תלתן ירוק אדומה\nQV Monza ...,<NA>
4,אלפא רומיאו,2011,5,ידנית,1400,בנזין,163000,לבן שנהב,פרטית,פרטית,נס ציונה - רחובות,רחובות,6,46000.0,אלפא רומיאו ג'ולייטה,2022-10-31,2022-10-31,רכב שמור מאוד דגם מילאנו עם גג פנורמי\nידני בי...,<NA>
5,אלפא רומיאו,2015,3,רובוטית,14000,בנזין,96000,NaN,NaN,NaN,חיפה וחוף הכרמל,חיפה,1,60000.0,אלפא רומיאו ג'ולייטה,2022-10-27,2022-10-27,אלפא רומיו ג'ולייטה\r\nרכב עבר רק במשפחה\r\n -...,<NA>
6,אלפא רומיאו,2010,3,ידנית,1400,בנזין,141000,NaN,NaN,NaN,עכו - נהריה,כפר יאסיף,1,32000.0,אלפא רומיאו מיטו,2022-09-27,2022-09-27,רכב במצב תצוגה אחרי טיפול 1.4 טורבו 155 כ״ס מנ...,<NA>
7,אלפא רומיאו,2011,4,ידנית,1400,בנזין,180,NaN,NaN,NaN,אשדוד - אשקלון,גן יבנה,1,15000.0,אלפא רומיאו מיטו,2022-09-20,2022-09-20,דגם ספורט ידני 6 הילוכים\r\nמולטימדיה הרכב במצ...,<NA>
8,אלפא רומיאו,2014,1,אוטומטית,1400,בנזין,0,NaN,NaN,NaN,מודיעין והסביבה,מודיעין מכבים רעות,1,50000.0,אלפא רומיאו ג'ולייטה,2022-07-21,2022-07-21,"פרטית, שמורה מאד\r\nטיפול כל 8000, טסט 3/23\r\...",<NA>
9,אלפא רומיאו,2011,2,רובוטית,3000,בנזין,140000,NaN,NaN,NaN,גליל ועמקים,נצרת עילית,1,36000.0,אלפא רומיאו 159,2022-07-02,2022-07-02,על הרכב\r\n159 2.2 סלסלפיד רמת גימור TI . אחרי...,<NA>


In [15]:
csv_file_path = 'car_info.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig', date_format='%Y-%m-%d')

print(f"DataFrame has been saved to {csv_file_path}")

from IPython.display import FileLink
FileLink(csv_file_path)

DataFrame has been saved to car_info.csv


C:\Users\itayh\Tasks\car_info.csv